# Summary

## Findings
* Many columns have missing data and four columns have more than 90% of values missing
* Four other columns had only one unique values

## Actions
* I will removed these columns that have more than 90% of values missing. Other columns with missing data will be handled later with imputation
* I removed columns 'experiences_offered', 'country_code', 'country', 'has_availability' due to having only one unique value
* Converted all the columns to appropriate data type
* Changed T/F columns to binary
* Cleaned the currency related fields, zipcodes, and percentage columns
* Removed listings with $500 or more daily price. Around 95% of listings are below this amount
* Added topic models from NLP work on the listing description
* Added zip code based metrics for income and population

<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Summary" data-toc-modified-id="Summary-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Summary</a></span><ul class="toc-item"><li><span><a href="#Findings" data-toc-modified-id="Findings-1.1"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>Findings</a></span></li><li><span><a href="#Actions" data-toc-modified-id="Actions-1.2"><span class="toc-item-num">1.2&nbsp;&nbsp;</span>Actions</a></span></li></ul></li><li><span><a href="#Functions" data-toc-modified-id="Functions-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Functions</a></span></li><li><span><a href="#Merge-results-from-Topic-Modeling" data-toc-modified-id="Merge-results-from-Topic-Modeling-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Merge results from Topic Modeling</a></span></li><li><span><a href="#Missing-Values" data-toc-modified-id="Missing-Values-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Missing Values</a></span></li><li><span><a href="#Remove-Single-Value-Columns" data-toc-modified-id="Remove-Single-Value-Columns-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>Remove Single Value Columns</a></span></li><li><span><a href="#Convert-Data-Types" data-toc-modified-id="Convert-Data-Types-6"><span class="toc-item-num">6&nbsp;&nbsp;</span>Convert Data Types</a></span></li><li><span><a href="#Misc-Cleaning" data-toc-modified-id="Misc-Cleaning-7"><span class="toc-item-num">7&nbsp;&nbsp;</span>Misc Cleaning</a></span></li><li><span><a href="#Add-external-zipcode-metrics" data-toc-modified-id="Add-external-zipcode-metrics-8"><span class="toc-item-num">8&nbsp;&nbsp;</span>Add external zipcode metrics</a></span></li><li><span><a href="#Calculate-distances" data-toc-modified-id="Calculate-distances-9"><span class="toc-item-num">9&nbsp;&nbsp;</span>Calculate distances</a></span></li><li><span><a href="#Export" data-toc-modified-id="Export-10"><span class="toc-item-num">10&nbsp;&nbsp;</span>Export</a></span></li></ul></div>

In [1]:
# Import needed libraries
import pandas as pd
import numpy as np
from geopy.distance import geodesic 

# Columns to be used during the project
cols = [
    'experiences_offered', 'host_response_time', 'host_response_rate',
    'host_acceptance_rate', 'host_is_superhost', 'host_listings_count',
    'host_has_profile_pic', 'host_identity_verified', 'country_code',
    'country', 'property_type', 'room_type', 'accommodates', 'bathrooms',
    'bedrooms', 'beds', 'bed_type', 'amenities', 'square_feet', 'price',
    'security_deposit', 'cleaning_fee', 'guests_included', 'extra_people',
    'minimum_nights', 'maximum_nights', 'has_availability', 'availability_30',
    'availability_60', 'availability_90', 'availability_365',
    'number_of_reviews', 'review_scores_rating', 'requires_license',
    'instant_bookable', 'cancellation_policy', 'reviews_per_month', 'zipcode',
    'id', 'latitude', 'longitude'
]

# Display all rows and columns
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)

df = pd.read_csv('data/sources/listings.csv.gz', low_memory=False, usecols=cols)

In [2]:
df.head()

,id,experiences_offered,host_response_time,host_response_rate,host_acceptance_rate,host_is_superhost,host_listings_count,host_has_profile_pic,host_identity_verified,zipcode,country_code,country,latitude,longitude,property_type,room_type,accommodates,bathrooms,bedrooms,beds,bed_type,amenities,square_feet,price,security_deposit,cleaning_fee,guests_included,extra_people,minimum_nights,maximum_nights,has_availability,availability_30,availability_60,availability_90,availability_365,number_of_reviews,review_scores_rating,requires_license,instant_bookable,cancellation_policy,reviews_per_month
0,16228948,none,a few days or more,25%,NaN,f,1.0,t,f,93510,US,United States,34.530350,-118.227088,House,Entire home/apt,10,7.0,5.0,5.0,Real Bed,"{""Wireless Internet"",""Air conditioning"",Pool,K...",NaN,"$3,000.00","$2,000.00",$200.00,1,$0.00,1,1125,NaN,27,53,82,352,0,NaN,f,t,strict,NaN
1,8909380,none,within an hour,100%,NaN,t,1.0,t,f,93550,US,United States,34.485431,-118.125380,House,Private room,2,1.0,1.0,1.0,Real Bed,{},NaN,$50.00,NaN,NaN,1,$0.00,1,1125,NaN,29,45,75,350,33,93.0,f,f,flexible,1.91
2,14078522,none,within an hour,100%,NaN,t,1.0,t,t,93510,US,United States,34.510130,-118.212407,House,Private room,2,1.0,1.0,1.0,Real Bed,"{""Wireless Internet"",""Air conditioning"",""Wheel...",NaN,$55.00,NaN,NaN,1,$10.00,1,1125,NaN,18,43,73,348,14,100.0,f,f,flexible,1.72
3,13006928,none,within an hour,100%,NaN,t,1.0,t,f,93510,US,United States,34.515486,-118.230742,Other,Entire home/apt,6,1.0,1.0,3.0,Real Bed,"{TV,""Wireless Internet"",""Air conditioning"",Poo...",NaN,$150.00,NaN,$35.00,1,$0.00,1,2,NaN,3,17,30,87,22,100.0,f,t,flexible,2.12
4,7898757,none,NaN,NaN,NaN,f,1.0,t,t,90007,US,United States,34.031419,-118.299016,Apartment,Private room,1,1.0,1.0,1.0,Real Bed,"{Internet,""Wireless Internet"",Kitchen,""Free pa...",NaN,$30.00,NaN,$5.00,1,$10.00,1,90,NaN,0,0,0,0,3,93.0,f,f,flexible,0.18


In [3]:
df.shape

(31253, 41)

# Functions

In [4]:
# Function to calculate missing values by column
def missing_values_table(df):
    # Total missing values
    mis_val = df.isnull().sum()

    # Percentage of missing values
    mis_val_percent = 100 * df.isnull().sum() / len(df)

    # Make a table with the results
    mis_val_table = pd.concat([mis_val, mis_val_percent], axis=1)

    # Rename the columns
    mis_val_table_ren_columns = mis_val_table.rename(columns={
        0: 'Missing Values',
        1: '% of Total Values'
    })

    # Sort the table by percentage of missing descending
    mis_val_table_ren_columns = mis_val_table_ren_columns[
        mis_val_table_ren_columns.iloc[:, 1] != 0].sort_values(
            '% of Total Values', ascending=False).round(1)

    # Print some summary information
    print("Your selected dataframe has " + str(df.shape[1]) + " columns and " +
          str(df.shape[0]) + " rows.\n"
          "There are " + str(mis_val_table_ren_columns.shape[0]) +
          " columns that have missing values.")

    # Return the dataframe with missing information
    return mis_val_table_ren_columns


# Removes specified columns
def drop_multiple_col(col_names_list, df):
    df.drop(col_names_list, axis=1, inplace=True)
    return df


# Function to remove columns with one singular value, no value to modeling process
def remove_constants(df):
    df_new = df.loc[:, df.apply(pd.Series.nunique) != 1]
    removed = list(df.loc[:, df.apply(pd.Series.nunique) == 1].columns)
    print(f'Removed columns: {removed}')
    return df_new


# Convert datatypes
def convert_types(df):

    # Iterate through each column
    for c in df:

        # Convert objects to category
        if (df[c].dtype == 'object') and (df[c].nunique() < df.shape[0]):
            df[c] = df[c].astype('category')

        # Booleans mapped to integers
        elif list(df[c].unique()) == [1, 0]:
            df[c] = df[c].astype(bool)

        # Float64 to float32
        elif df[c].dtype == float:
            df[c] = df[c].astype(np.float32)

        # Int64 to int32
        elif df[c].dtype == int:
            df[c] = df[c].astype(np.int32)

    return df

# Merge results from Topic Modeling

In [5]:
topics = pd.read_pickle('data/processed/listings_topics.pkl')
df = pd.merge(df, topics, on='id')

# Missing Values

In [6]:
missing = missing_values_table(df)
missing

Your selected dataframe has 42 columns and 31253 rows.
There are 17 columns that have missing values.


,Missing Values,% of Total Values
host_acceptance_rate,31253,100.0
has_availability,31253,100.0
square_feet,30869,98.8
security_deposit,14397,46.1
review_scores_rating,7527,24.1
reviews_per_month,7223,23.1
cleaning_fee,6974,22.3
host_response_rate,5561,17.8
host_response_time,5561,17.8
zipcode,395,1.3


In [7]:
# Setting condition to remove colums with 90% or more missing
remove = missing[missing['% of Total Values'] >= 90.0].index

df = drop_multiple_col(df[remove], df)

In [8]:
# I will substitute cleaning fee with zeros for nans
df['cleaning_fee'] = df.cleaning_fee.fillna(0)
df['host_response_rate'] = df.host_response_rate.fillna(0)

# Remove Single Value Columns

In [9]:
df = remove_constants(df)

Removed columns: ['experiences_offered', 'country_code', 'country']


# Convert Data Types

In [10]:
df = convert_types(df)

# Misc Cleaning

In [11]:
# Convert strings to integers for boolean values
bool_feat = [
    'host_identity_verified', 'host_has_profile_pic', 'host_is_superhost',
    'instant_bookable'
]
for b in (bool_feat):
    df[b] = df[b].map({'t': 1.0, 'f': 0.0})
    df[b] = df[b].astype('bool')

# Convert $ to float
convert = ('price', 'cleaning_fee', 'extra_people')
for con in (convert):
    df[con] = (df[con].replace('[\$,)]', '',
                               regex=True).replace('[(]', '-',
                                                   regex=True).astype(float))

# Remove % from host_response_rate column
df['host_response_rate'] = (df['host_response_rate'].replace(
    '[\%,)]', '', regex=True).replace('[(]', '-', regex=True).astype(float))

# Clean Zipcode
df['zipcode'] = df['zipcode'].str.replace(r'-\d+', '')

# # Drop rows with any missing zipcodes
df = df.dropna(how='any', subset=['zipcode'])

# Removing rows where zipcode is longer than 5 characters
df = df[df['zipcode'].map(len) == 5]
df['zipcode'] = df['zipcode'].astype(np.int32)

# Remove outliers with price
# 95%+ is below $500
df = df[df.price < 500]

# Remove all rows with zeros
df = df[df['price'] > 0.00]
df = df[df['accommodates'] != 0]
df = df[df['bedrooms'] != 0]
df = df[df['beds'] != 0]
df = df[df['zipcode'] != 0]
df = df[df['number_of_reviews'] != 0]

# Add external zipcode metrics

In [12]:
income = pd.read_excel('data/sources/median_home_value.xlsx').rename(
    columns={'Zip': 'zipcode'})
df = pd.merge(df, income, on='zipcode')

# Calculate distances

In [13]:
# I want to calculate the distance between the listing and multiple 
# locations in southern California
df = df.reset_index()

locations = pd.read_excel('data/sources/locations.xlsx')

locations_dist = []
for i,lat,lon in zip(df['id'], df['latitude'],df['longitude']):
    location = (lat,lon)
    dist_list=[]
    for p,l,n in zip(locations['location'], locations['lat'],locations['lon']):
        pop_locations = (l,n)
        dist = geodesic(location, pop_locations).mi
        dist_list.append(dist)
        
    locations_dist.append(dist_list)

In [14]:
location_df = pd.DataFrame(
    locations_dist,
    columns=[
        'distance_Disneyland', 'distance_Universal',
        'distance_Convention_Center', 'distance_LAX', 'distance_Downtown_LA',
        'distance_Seaworld', 'distance_Santa_Monica_beach',
        'distance_San_Diego', 'distance_Venice_beach',
        'distance_Griffith_Park', 'distance_Hollywood', 'distance_Long_beach',
        'distance_Ontario_airport', 'distance_Compton', 'distance_Rose_Bowl'
    ])

In [15]:
df = df.merge(location_df,left_index=True, right_index=True)

# Export

In [16]:
# Final look at the cleaned data
df = df.drop(columns=['City / Community','index','id'])
df.to_pickle('data/processed/listings_cleaned.pkl')
df.head()

,host_response_time,host_response_rate,host_is_superhost,host_listings_count,host_has_profile_pic,host_identity_verified,zipcode,latitude,longitude,property_type,room_type,accommodates,bathrooms,bedrooms,beds,bed_type,amenities,price,security_deposit,cleaning_fee,guests_included,extra_people,minimum_nights,maximum_nights,availability_30,availability_60,availability_90,availability_365,number_of_reviews,review_scores_rating,requires_license,instant_bookable,cancellation_policy,reviews_per_month,Topic,median_home_value,median_income,population,distance_Disneyland,distance_Universal,distance_Convention_Center,distance_LAX,distance_Downtown_LA,distance_Seaworld,distance_Santa_Monica_beach,distance_San_Diego,distance_Venice_beach,distance_Griffith_Park,distance_Hollywood,distance_Long_beach,distance_Ontario_airport,distance_Compton,distance_Rose_Bowl
0,within an hour,100.0,True,1.0,True,False,93550,34.485432,-118.125381,House,Private room,2,1.0,1.0,1.0,Real Bed,{},50.0,NaN,0.0,1,0.0,1,1125,29,45,75,350,33,93.0,f,False,flexible,1.91,6,278000,20222,75774,47.831340,27.240032,31.757775,40.730427,30.827716,129.380985,39.019592,134.346325,39.960522,27.162750,29.146583,49.875594,42.246084,40.869533,22.082232
1,within an hour,100.0,True,2.0,True,True,93550,34.556740,-118.063812,House,Private room,2,1.5,1.0,1.0,Real Bed,"{TV,""Cable TV"",Internet,""Wireless Internet"",""A...",32.0,NaN,10.0,1,5.0,1,1125,1,19,48,48,22,92.0,f,True,flexible,4.89,5,278000,20222,75774,51.928789,33.245785,37.478527,46.656835,36.517031,132.558790,45.058054,137.440999,45.971478,33.060207,35.105072,55.010913,43.570516,46.291467,27.506948
2,within an hour,100.0,True,2.0,True,True,93550,34.560383,-118.097412,House,Private room,2,1.0,1.0,1.0,Real Bed,"{Kitchen,""Free parking on premises"",""Smoking a...",24.0,NaN,0.0,1,8.0,1,1125,20,46,76,351,50,95.0,f,False,moderate,2.76,1,278000,20222,75774,52.516787,32.556593,37.161992,46.108002,36.228216,133.507963,44.231833,138.417538,45.259948,32.555840,34.510018,55.112044,44.965477,46.209716,27.409527
3,within a few hours,90.0,False,1.0,True,True,93550,34.557838,-118.084343,Apartment,Entire home/apt,4,2.0,2.0,3.0,Real Bed,"{TV,""Cable TV"",Internet,""Wireless Internet"",""A...",95.0,$150.00,0.0,1,0.0,6,150,19,21,21,272,12,96.0,f,False,flexible,0.38,1,278000,20222,75774,52.203890,32.744202,37.198973,46.241453,36.253740,133.064145,44.479491,137.963696,45.460138,32.667396,34.659456,54.987096,44.357639,46.155240,27.354661
4,NaN,0.0,True,1.0,True,True,93550,34.576488,-118.082581,House,Private room,2,1.5,1.0,1.0,Real Bed,"{TV,Internet,""Wireless Internet"",""Air conditio...",50.0,NaN,0.0,1,15.0,1,3,0,0,0,0,15,97.0,f,False,moderate,1.09,4,278000,20222,75774,53.450689,33.929663,38.460595,47.460506,37.519105,134.219215,45.620846,139.109282,46.637485,33.897307,35.868944,56.276564,45.302083,47.439612,28.638536
